In [2]:
from bs4 import BeautifulSoup
import requests
import json
import re

import pandas as pd

import os

# Get safety data from IIHS.org

In [178]:
def get_vehicle_info (soup):
    data_rows = []
    for row in soup.find_all('td', class_="Vehicle"):
        for td in row:
            iihs_url = url_base + td.get('href')
            names = [x.text for x in td.children]
            output = {}
            if ('Safety' in names[0]):
                make_model = names[1]
                style_years = ' '.join(names[2:])
                iihs_ranking = names[0]
            else:
                make_model = names[0]
                style_years = ' '.join(names[1:])
                iihs_ranking = ''
            (style, year) = style_years.split('|')
            style = style.strip()
            year_str = year.replace('models', '').strip()
            if len(year_str) == 4:
                year = int(year_str)
                other_notes = ""
            else:
                year = int(year_str[:4])
                other_notes = year_str[4:]
            make = make_model.split(' ')[0]
            model = ' '.join(make_model.split(' ')[1:])
            if (make=="Alfa"):
                make = "Alfa Romeo"
                model = model.replace("Romeo ", "")
            data_rows = data_rows + [{'make': make, 'model': model, 'style': style, 'year': year,
                                      "iihs_ranking": iihs_ranking, 'iihs_url': iihs_url,
                                      'other_notes': other_notes}]

    df = pd.DataFrame(data_rows)
    return (df)

In [179]:
def extract_table (url):
    page = requests.get(url)
    df_raw = pd.read_html(page.text)[0]
    soup = BeautifulSoup(page.text, 'html.parser')
    df_id = get_vehicle_info(soup).assign(iihs_type=os.path.basename(url))
    df = pd.concat([df_id, df_raw.drop(columns="Vehicle Name")], axis=1)
    return (df)

In [180]:
car_types = ["small-cars", "midsize-cars", "midsize-luxury-cars", "large-cars", "large-luxury-cars", 
             "small-suvs", "midsize-suvs", "midsize-luxury-suvs", "large-suvs", "minivans"]
url_base = "https://www.iihs.org"
urls = [url_base + "/ratings/class-summary/" + x for x in car_types]

In [181]:
print(urls[0])

https://www.iihs.org/ratings/class-summary/small-cars


In [182]:
car_table = pd.concat([extract_table(x) for x in urls]).reset_index(drop=True)

# Get fuel economy data from fueleconomy.gov

In [7]:
epadata_vehicles = pd.read_csv("https://www.fueleconomy.gov/feg/epadata/vehicles.csv.zip")

/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2878: DtypeWarning: Columns (70,71,72,73,74,76,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
pd.set_option('max_columns', None)
epadata_vehicles

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,14.167143,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Rear-Wheel Drive,9011,(FFS),-1,3000,0,Regular,Regular Gasoline,-1,-1,25,0.0,0,0.0,0.0,0.0,0.0,0,0,1,0,0,Alfa Romeo,Spider Veloce 2000,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,23.3333,0.0,35.0000,0.0,Two Seaters,1985,-3250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
1,27.046364,0.0,0.0,0.0,9,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,12.0,4.9,Rear-Wheel Drive,22020,(GUZZLER),-1,5750,0,Regular,Regular Gasoline,-1,-1,14,0.0,0,0.0,0.0,0.0,0.0,0,0,10,0,0,Ferrari,Testarossa,N,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,11.0000,0.0,19.0000,0.0,Two Seaters,1985,-17000,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
2,11.018889,0.0,0.0,0.0,23,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,329.148148,27,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,2100,(FFS),-1,2350,0,Regular,Regular Gasoline,-1,-1,33,0.0,0,0.0,0.0,0.0,0.0,19,77,100,0,0,Dodge,Charger,Y,False,0,0,0,0.0,0.0,0.0,0.0,Manual 5-spd,29.0000,0.0,47.0000,0.0,Subcompact Cars,1985,0,NaN,SIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
3,27.046364,0.0,0.0,0.0,10,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,807.909091,11,0.0,0,0.0,0.0,0.0,0.0,8.0,5.2,Rear-Wheel Drive,2850,NaN,-1,5750,0,Regular,Regular Gasoline,-1,-1,12,0.0,0,0.0,0.0,0.0,0.0,0,0,1000,0,0,Dodge,B150/B250 Wagon 2WD,N,False,0,0,0,0.0,0.0,0.0,0.0,Automatic 3-spd,12.2222,0.0,16.6667,0.0,Vans,1985,-17000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
4,15.658421,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,4-Wheel or All-Wheel Drive,66031,"(FFS,TRBO)",-1,3950,0,Premium,Premium Gasoline,-1,-1,23,0.0,0,0.0,0.0,0.0,0.0,0,0,10000,0,14,Subaru,Legacy AWD Turbo,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,21.0000,0.0,32.0000,0.0,Compact Cars,1993,-8000,NaN,NaN,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44821,13.523182,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,2900,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,0.0,0.0,0.0,0,0,9995,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Automatic 4-spd,24.0000,0.0,37.0000,0.0,Compact Cars,1993,-2750,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,Tue Jan 01 00:00:00 EST 2013,Tue Jan 01 00:00:00 EST 2013,NaN,0,0,0
44822,12.935217,0.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,386.391304,23,0.0,0,0.0,0.0,0.0,0.0,4.0,2.2,Front-Wheel Drive,66030,(FFS),-1,2750,0,Regular,Regular Gasoline,-1,-1,28,0.0,0,0.0,0.0,0.0,0.0,0,0,9996,0,14,Subaru,Legacy,N,False,0,90,0,0.0,0.0,0.0,0.0,Manual 5-spd,25.0000,0.0,39.0000,0.0,Compact

In [72]:
# def find_fuelecon_ids (make, model, year, db_full):
#     if (make=="Volvo"):
#         model = model.replace("Cross Country", "CC")
#     db = db_full[db_full["year"]>=(year-1)]
#     criterion1 = db["make"].str.match(make, case=False)
#     criterion2 = db["model"].str.startswith(model)
#     id_list = db[criterion1 & criterion2]["id"].to_list()
#     if (len(id_list)==0):
#         # criterion2 = True
#         if ((make=="BMW") & ('series' in model)):
#             criterion2 = db["model"].str.startswith(model.split(' ')[0])
#             model = model.replace(' '.join(model.split(' ')[0:2]), '')
#         if (' ' in model):
#             if not ((make=="BMW") & ('series' in model)):
#                 criterion2 = True
#             for x in model.split(' '):
#                 criterion2 = criterion2 & db["model"].str.contains(x, case=False)
#         if (make=="Mercedes-Benz"):
#             model = model.replace('-Class', '')
#             criterion2 = db["model"].str.replace('\d+', '').str.split().str.get(0).str.match(model)
#         id_list = db[criterion1 & criterion2]["id"].to_list()
#     return (id_list)

In [188]:
def find_fuelecon_ids (make, model, year, db_full):
    if (make=="Volvo"):
        model = model.replace("Cross Country", "CC")
    #db = db_full[db_full["year"]>=(year-1)]
    db = db_full[db_full["year"]==year]
    criterion1 = db["make"].str.match(make, case=False)
    criterion2 = db["model"].str.match(model, case=False)
    id_list = db[criterion1 & criterion2]["id"].to_list()
    if (len(id_list)==0):
        if ((make=="BMW") & ('series' in model)):
            criterion2 = db["model"].str.startswith(model.split(' ')[0])
            model = model.replace(' '.join(model.split(' ')[0:2]), '')
            if (' ' in model):
                for x in model.split(' '):
                    criterion2 = criterion2 & db["model"].str.contains(x, case=False)
        elif (make=="Mercedes-Benz"):
            model = model.replace('-Class', '')
            criterion2 = db["model"].str.replace('\d+', '').str.split().str.get(0).str.match(model)
        elif (make=="Volvo"):
            if (' ' in model):
                criterion2 = True
                for x in model.split(' '):
                    criterion2 = criterion2 & db["model"].str.contains(x, case=False)
        id_list = db[criterion1 & criterion2]["id"].to_list()
    return (id_list)

In [189]:
fueleconomy_ids = car_table[["make", "model", "year"]].\
apply(lambda x: find_fuelecon_ids(make=x[0], model=x[1], year=x[2], db_full=epadata_vehicles), axis=1)

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.


In [190]:
car_table_fuel = car_table.assign(fuelecon_ids=fueleconomy_ids)

In [191]:
car_table_fuel

,make,model,style,year,iihs_ranking,iihs_url,other_notes,iihs_type,Small overlap front,Moderate overlap front,Side,Roof,Head restraints & seats,Front crash prevention: vehicle-to-vehicle,Front crash prevention: vehicle-to-pedestrian,Headlights,Seat belt reminders,LATCH ease of use,fuelecon_ids
0,Honda,Civic,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/honda/civ...,,small-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,G,Not tested,G +,"[44131, 44132, 44133, 44134, 44545, 44546, 445..."
1,Honda,Insight,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/honda/ins...,,small-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,G,Not tested,G +,"[43947, 43948]"
2,Mazda,3,4-door hatchback,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/mazda/3-4...,,small-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,GA,Not tested,G +,"[45028, 45029, 45030, 45031, 45032, 45033, 450..."
3,Mazda,3,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/mazda/3-4...,,small-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,GA,Not tested,G +,"[45028, 45029, 45030, 45031, 45032, 45033, 450..."
4,Subaru,Crosstrek Hybrid,4-door wagon,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/subaru/cr...,,small-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,G,Not tested,G +,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Toyota,Sienna,minivan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/toyota/si...,,minivans,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Advanced Standard system,G,Not tested,G +,"[44551, 44552]"
165,Honda,Odyssey,minivan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/honda/ody...,,minivans,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,A,Not tested,G +,[43697]
166,Chrysler,Pacifica,minivan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/chrysler/...,,minivans,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Advanced Standard system,A,Not tested,A,"[44645, 44647, 44930]"
167,Kia,Carnival,minivan,2022,Top Safety Pick,https://www.iihs.org/ratings/vehicle/kia/carni...,built after March 2021,minivans,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,GP,Not tested,A,[43702]


In [205]:
car_table_fuel_subset = car_table_fuel.copy().loc[~ car_table_fuel["iihs_type"].isin(["small-cars", "midsize-cars"])]
len(car_table_fuel_subset)

129

In [206]:
car_table_fuel_subset["fuelecon_ids"].apply(len).value_counts()
# car_table_fuel["fuelecon_ids"].apply(len).value_counts()

2     30
3     27
4     21
1     20
6      9
5      9
7      4
8      4
12     2
10     1
15     1
0      1
Name: fuelecon_ids, dtype: int64

In [208]:
car_table_fuel_subset["num_fuelecon_ids"] = car_table_fuel_subset["fuelecon_ids"].apply(lambda x: len(x))

In [213]:
print(car_table_fuel_subset[car_table_fuel_subset["num_fuelecon_ids"] == 2]["iihs_url"].iloc[0])
car_table_fuel_subset[car_table_fuel_subset["num_fuelecon_ids"] == 2]

https://www.iihs.org/ratings/vehicle/volvo/s60-recharge-4-door-sedan/2022


,make,model,style,year,iihs_ranking,iihs_url,other_notes,iihs_type,Small overlap front,Moderate overlap front,Side,Roof,Head restraints & seats,Front crash prevention: vehicle-to-vehicle,Front crash prevention: vehicle-to-pedestrian,Headlights,Seat belt reminders,LATCH ease of use,fuelecon_ids,num_fuelecon_ids
44,Volvo,S60 Recharge,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/volvo/s60...,,midsize-luxury-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,G,Not tested,A,"[44266, 45197]",2
46,Lexus,ES 350,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/lexus/es-...,All ratings apply to both the ES 350 and ES ...,midsize-luxury-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Advanced Standard system,G,Not tested,G,"[44454, 44455]",2
49,Audi,A5 Sportback,4-door hatchback,2022,Top Safety Pick,https://www.iihs.org/ratings/vehicle/audi/a5-s...,,midsize-luxury-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,GAM,Not tested,G,"[44118, 44120]",2
50,BMW,2 series Gran Coupe,4-door sedan,2022,Top Safety Pick,https://www.iihs.org/ratings/vehicle/bmw/2-ser...,,midsize-luxury-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Advanced Standard system,AM,Not tested,G,"[44179, 44180]",2
65,Audi,A7,4-door hatchback,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/audi/a7-4...,,large-luxury-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,A,Not tested,G,"[44125, 44158]",2
68,Volvo,S90 Recharge,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/volvo/s90...,,large-luxury-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Advanced Standard system,A,Not tested,A,"[44267, 45198]",2
73,Mazda,CX-30,4-door SUV,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/mazda/cx-...,,small-suvs,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,G,Not tested,G +,"[45114, 45115]",2
76,Ford,Bronco Sport,4-door SUV,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/ford/bron...,,small-suvs,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,G,Not tested,A,"[44816, 44817]",2
78,Mazda,CX-5,4-door SUV,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/mazda/cx-...,,small-suvs,G Driver-side G Passenger-side,G,G original test G updated test,G,G,Superior Standard system,Advanced Standard system,GA,M,A,"[45003, 45004]",2
93,Audi,Q3,4-door SUV,2022,,https://www.iihs.org/ratings/vehicle/audi/q3-4...,,small-suvs,G Driver-side G Passenger-side,G,G original test A updated test,G,G,Superior Standard system,Basic Standard system,GP,P,G +,"[44147, 44148]",2


In [127]:
#epadata_vehicles[(epadata_vehicles["make"] == "Volvo") & (epadata_vehicles["model"].str.startswith("S60"))]["model"].unique()

array(['S60', 'S60 AWD', 'S60 FWD', 'S60 R AWD', 'S60 PoleStar AWD',
       'S60 Inscription AWD', 'S60 CC AWD', 'S60 Inscription FWD',
       'S60 Polestar AWD', 'S60 AWD PHEV', 'S60 B5', 'S60 B5 AWD',
       'S60 T8 AWD Recharge', 'S60 T8 AWD Recharge ext. Range'],
      dtype=object)

In [118]:
# car_table[(car_table["make"] == "Volvo") & (car_table["model"].str.startswith("S60"))]["model"].unique()

array(['S60', 'S60 Recharge'], dtype=object)

In [212]:
tmp_ids = car_table_fuel_subset[car_table_fuel_subset["num_fuelecon_ids"] == 8]["fuelecon_ids"].iloc[0]
epadata_vehicles[epadata_vehicles["id"].isin(tmp_ids)]

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
38439,2.486761,3.165,0.0,4.5,40,40.3669,105,104.5381,0.0,32.0,0.721,72,-1,0.0,72.0,38,38.0000,94,94.0002,36.0,0.0,0.693,4.0,2.5,All-Wheel Drive,106,SIDI & PFI,10,1650,0,Regular Gas and Electricity,Regular Gasoline,10,-1,36,36.0000,84,83.6892,0.0,40.0,0.654,0,0,44984,0,0,Toyota,RAV4 Prime 4WD,N,True,0,0,0,0.0,46.22,0.0,37.13,Automatic (AV-S6),54.8425,149.3,52.3939,119.6,Small Sport Utility Vehicle 4WD,2022,6750,NaN,NaN,NaN,NaN,Plug-in Hybrid,Electricity,42,40 and 134 kW AC Induction,TYX,NaN,0.0,NaN,Thu Dec 16 00:00:00 EST 2021,Fri Mar 25 00:00:00 EDT 2022,Y,72,58,65
38447,9.917000,0.000,0.0,0.0,27,26.7869,0,0.0000,0.0,0.0,0.000,296,-1,0.0,296.0,30,29.8882,0,0.0000,0.0,0.0,0.000,4.0,2.5,Front-Wheel Drive,74,SIDI & PFI,6,2100,0,Regular,Regular Gasoline,6,-1,35,34.8146,0,0.0000,0.0,0.0,0.000,0,0,44991,0,0,Toyota,RAV4,N,False,0,0,0,0.0,0.00,0.0,0.00,Automatic (S8),34.9000,0.0,50.7000,0.0,Small Sport Utility Vehicle 2WD,2022,1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TYX,NaN,0.0,NaN,Mon Dec 20 00:00:00 EST 2021,Wed Jan 19 00:00:00 EST 2022,N,0,0,0
38448,9.917000,0.000,0.0,0.0,27,27.4768,0,0.0000,0.0,0.0,0.000,292,-1,0.0,292.0,30,30.2692,0,0.0000,0.0,0.0,0.000,4.0,2.5,Front-Wheel Drive,75,SIDI & PFI; Stop-Start,6,2100,0,Regular,Regular Gasoline,6,-1,35,34.5621,0,0.0000,0.0,0.0,0.000,0,0,44992,0,0,Toyota,RAV4,N,False,0,0,0,0.0,0.00,0.0,0.00,Automatic (S8),35.9127,0.0,50.2867,0.0,Small Sport Utility Vehicle 2WD,2022,1250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TYX,NaN,0.0,NaN,Mon Dec 20 00:00:00 EST 2021,Wed Jan 19 00:00:00 EST 2022,Y,0,0,0
38464,10.625357,0.000,0.0,0.0,25,24.7171,0,0.0000,0.0,0.0,0.000,319,-1,0.0,319.0,28,27.8551,0,0.0000,0.0,0.0,0.000,4.0,2.5,All-Wheel Drive,76,SIDI & PFI,6,2250,0,Regular,Regular Gasoline,6,-1,33,32.9713,0,0.0000,0.0,0.0,0.000,0,0,45005,0,0,Toyota,RAV4 AWD,N,False,0,0,0,0.0,0.00,0.0,0.00,Automatic (S8),31.9000,0.0,47.7000,0.0,Small Sport Utility Vehicle 4WD,2022,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TYX,NaN,0.0,NaN,Mon Dec 20 00:00:00 EST 2021,Wed Jan 19 00:00:00 EST 2022,N,0,0,0
38465,10.258966,0.000,0.0,0.0,27,26.7185,0,0.0000,0.0,0.0,0.000,301,-1,0.0,301.0,29,29.2767,0,0.0000,0.0,0.0,0.000,4.0,2.5,All-Wheel Drive,77,SIDI & PFI; Stop-Start,6,2200,0,Regular,Regular Gasoline,6,-1,33,33.1568,0,0.0000,0.0,0.0,0.000,0,0,45006,0,0,Toyota,RAV4 AWD,N,False,0,0,0,0.0,0.00,0.0,0.00,Automatic (S8),34.8000,0.0,48.0000,0.0,Small Sport Utility Vehicle 4WD,2022,750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TYX,NaN,0.0,NaN,Mon Dec 20 00:00:00 EST 2021,Wed Jan 19 00:00:00 EST 2022,Y,0,0,0
38466,10.625357,0.000,0.0,0.0,25,24.8563,0,0.0000,0.0,0.0,0.000,323,-1,0.0,323.0,28,27.5407,0,0.0000,0.0,0.0,0.000,4.0,2.5,All-Wheel Drive,78,SIDI & PFI,6,2250,0,Regular,Regular Gasoline,6,-1,32,31.7289,0,0.0000,0.0,0.0,0.000,0,0,45007,0,0,Toyota,RAV4 AWD TRD OFFROAD,N,False,0,0,0,0.0,0.00,0.0,0.00,Automatic (S8),32.1000,0.0,45.7000,0.0,Small Sport Utility Vehicle 4WD,2022,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TYX,NaN,0.0,NaN,Mon Dec 20 00:00:00 EST 2021,Wed Jan 19 00:00:00 EST 2022,N,0,0,0
38467,9.917000,0.000,0.0,0.0,27,27.1279,0,0.0000,0.0,0.0,0.000,296,-1,0.0,296.0,30,29.8272,0,0.0000,0.0,0.0,0.000,4.0,2.5,All-Wheel Drive,79,SIDI & PFI,6,2100,0,Regular,Regular Gasoline,6,-1,34,33.9569,0,0.0000,0.0,0.0,0.000,0,0,45008,0,0,Toyota,RAV4 AWD LE,N,False,0,0,0,0.0,0.00,

# Group by and aggregate

In [240]:
from numpy import dtype
import numpy as np

In [256]:
epadata_vehicles_numerical_cols =\
    list(epadata_vehicles.columns[epadata_vehicles.dtypes.isin([dtype('int64'), dtype('float64')])])
epadata_vehicles_numerical_cols.remove('year')

In [293]:
def compute_stats(stat_name, numpy_func):
    rename_cols = dict([(c, c + "_" + stat_name) for c in epadata_vehicles_numerical_cols])
    epadata_vehicles_tmp = epadata_vehicles[epadata_vehicles['year'] >= 2021].copy().rename(columns=rename_cols)
    return epadata_vehicles_tmp.groupby(['make', 'model', 'year', 'fuelType']).aggregate(numpy_func)

In [312]:
epadata_vehicles_stats = pd.concat([compute_stats('min', np.nanmin), compute_stats('max', np.nanmax),
                                    compute_stats('mean', np.nanmean), compute_stats('med', np.nanmedian)],
                                   axis=1)

/opt/miniconda3/lib/python3.7/site-packages/pandas/core/apply.py:169: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  return getattr(obj, f)()
/opt/miniconda3/lib/python3.7/site-packages/pandas/core/apply.py:169: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  return getattr(obj, f)()


In [313]:
epadata_vehicles_stats.reset_index(inplace=True)

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """Entry point for launching an IPython kernel.


In [314]:
car_table_fuel[car_table_fuel[['make', 'model', 'year']].duplicated()]

,make,model,style,year,iihs_ranking,iihs_url,other_notes,iihs_type,Small overlap front,Moderate overlap front,Side,Roof,Head restraints & seats,Front crash prevention: vehicle-to-vehicle,Front crash prevention: vehicle-to-pedestrian,Headlights,Seat belt reminders,LATCH ease of use,fuelecon_ids
3,Mazda,3,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/mazda/3-4...,,small-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,GA,Not tested,G +,"[45028, 45029, 45030, 45031, 45032, 45033, 450..."
5,Honda,Civic,4-door hatchback,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/honda/civ...,,small-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,G,Not tested,Not rated,"[44131, 44132, 44133, 44134, 44545, 44546, 445..."
7,Toyota,Corolla,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/toyota/co...,,small-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,AM,Not tested,G +,"[44070, 44071, 44072, 44073, 44074, 44075, 440..."
13,Subaru,Impreza,4-door sedan,2022,,https://www.iihs.org/ratings/vehicle/subaru/im...,,small-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Optional system,Superior Optional system,M,Not tested,G +,"[44059, 44060, 44061, 44062, 44063, 44064, 44065]"


In [315]:
merged_all_the_data_yeah = pd.merge(car_table_fuel, epadata_vehicles_stats, how='left', on=['make', 'model', 'year'])
merged_all_the_data_yeah

,make,model,style,year,iihs_ranking,iihs_url,other_notes,iihs_type,Small overlap front,Moderate overlap front,Side,Roof,Head restraints & seats,Front crash prevention: vehicle-to-vehicle,Front crash prevention: vehicle-to-pedestrian,Headlights,Seat belt reminders,LATCH ease of use,fuelecon_ids,fuelType,barrels08_min,barrelsA08_min,charge120_min,charge240_min,city08_min,city08U_min,cityA08_min,cityA08U_min,cityCD_min,cityE_min,cityUF_min,co2_min,co2A_min,co2TailpipeAGpm_min,co2TailpipeGpm_min,comb08_min,comb08U_min,combA08_min,combA08U_min,combE_min,combinedCD_min,combinedUF_min,cylinders_min,displ_min,drive,engId_min,feScore_min,fuelCost08_min,fuelCostA08_min,fuelType1,ghgScore_min,ghgScoreA_min,highway08_min,highway08U_min,highwayA08_min,highwayA08U_min,highwayCD_min,highwayE_min,highwayUF_min,hlv_min,hpv_min,id_min,lv2_min,lv4_min,mpgData,phevBlended,pv2_min,pv4_min,range_min,rangeCity_min,rangeCityA_min,rangeHwy_min,rangeHwyA_min,trany,UCity_min,UCityA_min,UHighway_min,UHighwayA_min,VClass,youSaveSpend_min,trans_dscr,fuelType2,rangeA,mfrCode,c240Dscr,charge240b_min,c240bDscr,createdOn,modifiedOn,startStop,phevCity_min,phevHwy_min,phevComb_min,barrels08_max,barrelsA08_max,charge120_max,charge240_max,city08_max,city08U_max,cityA08_max,cityA08U_max,cityCD_max,cityE_max,cityUF_max,co2_max,co2A_max,co2TailpipeAGpm_max,co2TailpipeGpm_max,comb08_max,comb08U_max,combA08_max,combA08U_max,combE_max,combinedCD_max,combinedUF_max,cylinders_max,displ_max,drive,engId_max,feScore_max,fuelCost08_max,fuelCostA08_max,fuelType1,ghgScore_max,ghgScoreA_max,highway08_max,highway08U_max,highwayA08_max,highwayA08U_max,highwayCD_max,highwayE_max,highwayUF_max,hlv_max,hpv_max,id_max,lv2_max,lv4_max,mpgData,phevBlended,pv2_max,pv4_max,range_max,rangeCity_max,rangeCityA_max,rangeHwy_max,rangeHwyA_max,trany,UCity_max,UCityA_max,UHighway_max,UHighwayA_max,VClass,youSaveSpend_max,trans_dscr,fuelType2,rangeA,mfrCode,c240Dscr,charge240b_max,c240bDscr,createdOn,modifiedOn,startStop,phevCity_max,phevHwy_max,phevComb_max,barrels08_mean,barrelsA08_mean,charge120_mean,charge240_mean,city08_mean,city08U_mean,cityA08_mean,cityA08U_mean,cityCD_mean,cityE_mean,cityUF_mean,co2_mean,co2A_mean,co2TailpipeAGpm_mean,co2TailpipeGpm_mean,comb08_mean,comb08U_mean,combA08_mean,combA08U_mean,combE_mean,combinedCD_mean,combinedUF_mean,cylinders_mean,displ_mean,engId_mean,feScore_mean,fuelCost08_mean,fuelCostA08_mean,ghgScore_mean,ghgScoreA_mean,highway08_mean,highway08U_mean,highwayA08_mean,highwayA08U_mean,highwayCD_mean,highwayE_mean,highwayUF_mean,hlv_mean,hpv_mean,id_mean,lv2_mean,lv4_mean,phevBlended,pv2_mean,pv4_mean,range_mean,rangeCity_mean,rangeCityA_mean,rangeHwy_mean,rangeHwyA_mean,UCity_mean,UCityA_mean,UHighway_mean,UHighwayA_mean,youSaveSpend_mean,charge240b_mean,phevCity_mean,phevHwy_mean,phevComb_mean,barrels08_med,barrelsA08_med,charge120_med,charge240_med,city08_med,city08U_med,cityA08_med,cityA08U_med,cityCD_med,cityE_med,cityUF_med,co2_med,co2A_med,co2TailpipeAGpm_med,co2TailpipeGpm_med,comb08_med,comb08U_med,combA08_med,combA08U_med,combE_med,combinedCD_med,combinedUF_med,cylinders_med,displ_med,engId_med,feScore_med,fuelCost08_med,fuelCostA08_med,ghgScore_med,ghgScoreA_med,highway08_med,highway08U_med,highwayA08_med,highwayA08U_med,highwayCD_med,highwayE_med,highwayUF_med,hlv_med,hpv_med,id_med,lv2_med,lv4_med,phevBlended,pv2_med,pv4_med,range_med,rangeCity_med,rangeCityA_med,rangeHwy_med,rangeHwyA_med,UCity_med,UCityA_med,UHighway_med,UHighwayA_med,youSaveSpend_med,charge240b_med,phevCity_med,phevHwy_med,phevComb_med
0,Honda,Civic,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/honda/civ...,,small-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,G,Not tested,G +,"[44131, 44132, 44133, 44134, 44545, 44546, 445...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [323]:
epadata_vehicles_stats[(epadata_vehicles_stats['make']=='Acura') & (epadata_vehicles_stats['model']=='TLX')]

,make,model,year,fuelType,barrels08_min,barrelsA08_min,charge120_min,charge240_min,city08_min,city08U_min,cityA08_min,cityA08U_min,cityCD_min,cityE_min,cityUF_min,co2_min,co2A_min,co2TailpipeAGpm_min,co2TailpipeGpm_min,comb08_min,comb08U_min,combA08_min,combA08U_min,combE_min,combinedCD_min,combinedUF_min,cylinders_min,displ_min,drive,engId_min,feScore_min,fuelCost08_min,fuelCostA08_min,fuelType1,ghgScore_min,ghgScoreA_min,highway08_min,highway08U_min,highwayA08_min,highwayA08U_min,highwayCD_min,highwayE_min,highwayUF_min,hlv_min,hpv_min,id_min,lv2_min,lv4_min,mpgData,phevBlended,pv2_min,pv4_min,range_min,rangeCity_min,rangeCityA_min,rangeHwy_min,rangeHwyA_min,trany,UCity_min,UCityA_min,UHighway_min,UHighwayA_min,VClass,youSaveSpend_min,trans_dscr,fuelType2,rangeA,mfrCode,c240Dscr,charge240b_min,c240bDscr,createdOn,modifiedOn,startStop,phevCity_min,phevHwy_min,phevComb_min,barrels08_max,barrelsA08_max,charge120_max,charge240_max,city08_max,city08U_max,cityA08_max,cityA08U_max,cityCD_max,cityE_max,cityUF_max,co2_max,co2A_max,co2TailpipeAGpm_max,co2TailpipeGpm_max,comb08_max,comb08U_max,combA08_max,combA08U_max,combE_max,combinedCD_max,combinedUF_max,cylinders_max,displ_max,drive,engId_max,feScore_max,fuelCost08_max,fuelCostA08_max,fuelType1,ghgScore_max,ghgScoreA_max,highway08_max,highway08U_max,highwayA08_max,highwayA08U_max,highwayCD_max,highwayE_max,highwayUF_max,hlv_max,hpv_max,id_max,lv2_max,lv4_max,mpgData,phevBlended,pv2_max,pv4_max,range_max,rangeCity_max,rangeCityA_max,rangeHwy_max,rangeHwyA_max,trany,UCity_max,UCityA_max,UHighway_max,UHighwayA_max,VClass,youSaveSpend_max,trans_dscr,fuelType2,rangeA,mfrCode,c240Dscr,charge240b_max,c240bDscr,createdOn,modifiedOn,startStop,phevCity_max,phevHwy_max,phevComb_max,barrels08_mean,barrelsA08_mean,charge120_mean,charge240_mean,city08_mean,city08U_mean,cityA08_mean,cityA08U_mean,cityCD_mean,cityE_mean,cityUF_mean,co2_mean,co2A_mean,co2TailpipeAGpm_mean,co2TailpipeGpm_mean,comb08_mean,comb08U_mean,combA08_mean,combA08U_mean,combE_mean,combinedCD_mean,combinedUF_mean,cylinders_mean,displ_mean,engId_mean,feScore_mean,fuelCost08_mean,fuelCostA08_mean,ghgScore_mean,ghgScoreA_mean,highway08_mean,highway08U_mean,highwayA08_mean,highwayA08U_mean,highwayCD_mean,highwayE_mean,highwayUF_mean,hlv_mean,hpv_mean,id_mean,lv2_mean,lv4_mean,phevBlended,pv2_mean,pv4_mean,range_mean,rangeCity_mean,rangeCityA_mean,rangeHwy_mean,rangeHwyA_mean,UCity_mean,UCityA_mean,UHighway_mean,UHighwayA_mean,youSaveSpend_mean,charge240b_mean,phevCity_mean,phevHwy_mean,phevComb_mean,barrels08_med,barrelsA08_med,charge120_med,charge240_med,city08_med,city08U_med,cityA08_med,cityA08U_med,cityCD_med,cityE_med,cityUF_med,co2_med,co2A_med,co2TailpipeAGpm_med,co2TailpipeGpm_med,comb08_med,comb08U_med,combA08_med,combA08U_med,combE_med,combinedCD_med,combinedUF_med,cylinders_med,displ_med,engId_med,feScore_med,fuelCost08_med,fuelCostA08_med,ghgScore_med,ghgScoreA_med,highway08_med,highway08U_med,highwayA08_med,highwayA08U_med,highwayCD_med,highwayE_med,highwayUF_med,hlv_med,hpv_med,id_med,lv2_med,lv4_med,phevBlended,pv2_med,pv4_med,range_med,rangeCity_med,rangeCityA_med,rangeHwy_med,rangeHwyA_med,UCity_med,UCityA_med,UHighway_med,UHighwayA_med,youSaveSpend_med,charge240b_med,phevCity_med,phevHwy_med,phevComb_med


In [316]:
merged_all_the_data_yeah[(merged_all_the_data_yeah['make']=='Acura') & (merged_all_the_data_yeah['model']=='TLX')]

,make,model,style,year,iihs_ranking,iihs_url,other_notes,iihs_type,Small overlap front,Moderate overlap front,Side,Roof,Head restraints & seats,Front crash prevention: vehicle-to-vehicle,Front crash prevention: vehicle-to-pedestrian,Headlights,Seat belt reminders,LATCH ease of use,fuelecon_ids,fuelType,barrels08_min,barrelsA08_min,charge120_min,charge240_min,city08_min,city08U_min,cityA08_min,cityA08U_min,cityCD_min,cityE_min,cityUF_min,co2_min,co2A_min,co2TailpipeAGpm_min,co2TailpipeGpm_min,comb08_min,comb08U_min,combA08_min,combA08U_min,combE_min,combinedCD_min,combinedUF_min,cylinders_min,displ_min,drive,engId_min,feScore_min,fuelCost08_min,fuelCostA08_min,fuelType1,ghgScore_min,ghgScoreA_min,highway08_min,highway08U_min,highwayA08_min,highwayA08U_min,highwayCD_min,highwayE_min,highwayUF_min,hlv_min,hpv_min,id_min,lv2_min,lv4_min,mpgData,phevBlended,pv2_min,pv4_min,range_min,rangeCity_min,rangeCityA_min,rangeHwy_min,rangeHwyA_min,trany,UCity_min,UCityA_min,UHighway_min,UHighwayA_min,VClass,youSaveSpend_min,trans_dscr,fuelType2,rangeA,mfrCode,c240Dscr,charge240b_min,c240bDscr,createdOn,modifiedOn,startStop,phevCity_min,phevHwy_min,phevComb_min,barrels08_max,barrelsA08_max,charge120_max,charge240_max,city08_max,city08U_max,cityA08_max,cityA08U_max,cityCD_max,cityE_max,cityUF_max,co2_max,co2A_max,co2TailpipeAGpm_max,co2TailpipeGpm_max,comb08_max,comb08U_max,combA08_max,combA08U_max,combE_max,combinedCD_max,combinedUF_max,cylinders_max,displ_max,drive,engId_max,feScore_max,fuelCost08_max,fuelCostA08_max,fuelType1,ghgScore_max,ghgScoreA_max,highway08_max,highway08U_max,highwayA08_max,highwayA08U_max,highwayCD_max,highwayE_max,highwayUF_max,hlv_max,hpv_max,id_max,lv2_max,lv4_max,mpgData,phevBlended,pv2_max,pv4_max,range_max,rangeCity_max,rangeCityA_max,rangeHwy_max,rangeHwyA_max,trany,UCity_max,UCityA_max,UHighway_max,UHighwayA_max,VClass,youSaveSpend_max,trans_dscr,fuelType2,rangeA,mfrCode,c240Dscr,charge240b_max,c240bDscr,createdOn,modifiedOn,startStop,phevCity_max,phevHwy_max,phevComb_max,barrels08_mean,barrelsA08_mean,charge120_mean,charge240_mean,city08_mean,city08U_mean,cityA08_mean,cityA08U_mean,cityCD_mean,cityE_mean,cityUF_mean,co2_mean,co2A_mean,co2TailpipeAGpm_mean,co2TailpipeGpm_mean,comb08_mean,comb08U_mean,combA08_mean,combA08U_mean,combE_mean,combinedCD_mean,combinedUF_mean,cylinders_mean,displ_mean,engId_mean,feScore_mean,fuelCost08_mean,fuelCostA08_mean,ghgScore_mean,ghgScoreA_mean,highway08_mean,highway08U_mean,highwayA08_mean,highwayA08U_mean,highwayCD_mean,highwayE_mean,highwayUF_mean,hlv_mean,hpv_mean,id_mean,lv2_mean,lv4_mean,phevBlended,pv2_mean,pv4_mean,range_mean,rangeCity_mean,rangeCityA_mean,rangeHwy_mean,rangeHwyA_mean,UCity_mean,UCityA_mean,UHighway_mean,UHighwayA_mean,youSaveSpend_mean,charge240b_mean,phevCity_mean,phevHwy_mean,phevComb_mean,barrels08_med,barrelsA08_med,charge120_med,charge240_med,city08_med,city08U_med,cityA08_med,cityA08U_med,cityCD_med,cityE_med,cityUF_med,co2_med,co2A_med,co2TailpipeAGpm_med,co2TailpipeGpm_med,comb08_med,comb08U_med,combA08_med,combA08U_med,combE_med,combinedCD_med,combinedUF_med,cylinders_med,displ_med,engId_med,feScore_med,fuelCost08_med,fuelCostA08_med,ghgScore_med,ghgScoreA_med,highway08_med,highway08U_med,highwayA08_med,highwayA08U_med,highwayCD_med,highwayE_med,highwayUF_med,hlv_med,hpv_med,id_med,lv2_med,lv4_med,phevBlended,pv2_med,pv4_med,range_med,rangeCity_med,rangeCityA_med,rangeHwy_med,rangeHwyA_med,UCity_med,UCityA_med,UHighway_med,UHighwayA_med,youSaveSpend_med,charge240b_med,phevCity_med,phevHwy_med,phevComb_med
42,Acura,TLX,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/acura/tlx...,,midsize-luxury-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,G,Not tested,G +,"[44617, 44618, 44619, 44620, 44621, 44622]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [317]:
car_table_fuel[(car_table_fuel['make']=='Acura') & (car_table_fuel['model']=='TLX')]

,make,model,style,year,iihs_ranking,iihs_url,other_notes,iihs_type,Small overlap front,Moderate overlap front,Side,Roof,Head restraints & seats,Front crash prevention: vehicle-to-vehicle,Front crash prevention: vehicle-to-pedestrian,Headlights,Seat belt reminders,LATCH ease of use,fuelecon_ids
40,Acura,TLX,4-door sedan,2022,Top Safety Pick +,https://www.iihs.org/ratings/vehicle/acura/tlx...,,midsize-luxury-cars,G Driver-side G Passenger-side,G,G original test,G,G,Superior Standard system,Superior Standard system,G,Not tested,G +,"[44617, 44618, 44619, 44620, 44621, 44622]"


In [318]:
epadata_vehicles[(epadata_vehicles['make'].str.match('Acura', case=False)) &
                 (epadata_vehicles['model'].str.match('TLX', case=False))]

,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,VClass,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
27812,10.625357,0.0,0.0,0.0,24,24.0000,0,0.0,0.0,0.0,0.0,321,-1,0.0,321.0,28,28.0000,0,0.0,0.0,0.0,0.0,4.0,2.4,Front-Wheel Drive,18,SIDI,7,2650,0,Premium,Premium Gasoline,7,-1,35,35.0000,0,0.0,0.0,0.0,0.0,0,0,35329,0,14,Acura,TLX,Y,False,0,93,0,0.0,0.0,0.0,0.0,Automatic (AM8),31.9972,0.0,50.4460,0.0,Compact Cars,2015,-1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HNX,NaN,0.0,NaN,Thu Jul 31 00:00:00 EDT 2014,Wed Apr 05 00:00:00 EDT 2017,N,0,0,0
27986,11.900400,0.0,0.0,0.0,21,21.0000,0,0.0,0.0,0.0,0.0,358,-1,0.0,358.0,25,25.0000,0,0.0,0.0,0.0,0.0,6.0,3.5,Front-Wheel Drive,19,SIDI,6,3000,0,Premium,Premium Gasoline,6,-1,34,34.0000,0,0.0,0.0,0.0,0.0,0,0,35487,0,14,Acura,TLX,Y,False,0,93,0,0.0,0.0,0.0,0.0,Automatic (S9),27.9000,0.0,49.5998,0.0,Compact Cars,2015,-3250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HNX,NaN,0.0,NaN,Tue Aug 12 00:00:00 EDT 2014,Mon Sep 26 00:00:00 EDT 2016,N,0,0,0
28127,11.900400,0.0,0.0,0.0,21,21.3582,0,0.0,0.0,0.0,0.0,360,-1,0.0,360.0,25,24.9886,0,0.0,0.0,0.0,0.0,6.0,3.5,All-Wheel Drive,20,SIDI,6,3000,0,Premium,Premium Gasoline,6,-1,31,31.0000,0,0.0,0.0,0.0,0.0,0,0,35613,0,14,Acura,TLX AWD,Y,False,0,93,0,0.0,0.0,0.0,0.0,Automatic (S9),27.1499,0.0,45.3998,0.0,Compact Cars,2015,-3250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HNX,NaN,0.0,NaN,Thu Sep 04 00:00:00 EDT 2014,Mon Sep 26 00:00:00 EDT 2016,Y,0,0,0
29585,10.625357,0.0,0.0,0.0,24,24.0000,0,0.0,0.0,0.0,0.0,321,-1,0.0,321.0,28,28.0000,0,0.0,0.0,0.0,0.0,4.0,2.4,Front-Wheel Drive,31,SIDI,7,2650,0,Premium,Premium Gasoline,7,-1,35,35.0000,0,0.0,0.0,0.0,0.0,0,0,36948,0,14,Acura,TLX 2WD,N,False,0,93,0,0.0,0.0,0.0,0.0,Automatic (AM-S8),31.9972,0.0,50.4460,0.0,Compact Cars,2016,-1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HNX,NaN,0.0,NaN,Fri Sep 25 00:00:00 EDT 2015,Wed Apr 05 00:00:00 EDT 2017,N,0,0,0
29586,11.900400,0.0,0.0,0.0,21,21.0000,0,0.0,0.0,0.0,0.0,358,-1,0.0,358.0,25,25.0000,0,0.0,0.0,0.0,0.0,6.0,3.5,Front-Wheel Drive,32,SIDI,6,3000,0,Premium,Premium Gasoline,6,-1,34,34.0000,0,0.0,0.0,0.0,0.0,0,0,36949,0,14,Acura,TLX 2WD,N,False,0,93,0,0.0,0.0,0.0,0.0,Automatic (S9),27.9000,0.0,49.5998,0.0,Compact Cars,2016,-3250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HNX,NaN,0.0,NaN,Fri Sep 25 00:00:00 EDT 2015,Mon Sep 26 00:00:00 EDT 2016,N,0,0,0
29588,11.900400,0.0,0.0,0.0,21,21.3582,0,0.0,0.0,0.0,0.0,360,-1,0.0,360.0,25,24.9886,0,0.0,0.0,0.0,0.0,6.0,3.5,All-Wheel Drive,33,SIDI,6,3000,0,Premium,Premium Gasoline,6,-1,31,31.0000,0,0.0,0.0,0.0,0.0,0,0,36950,0,14,Acura,TLX 4WD,N,False,0,93,0,0.0,0.0,0.0,0.0,Automatic (S9),27.1499,0.0,45.3998,0.0,Compact Cars,2016,-3250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HNX,NaN,0.0,NaN,Fri Sep 25 00:00:00 EDT 2015,Mon Sep 26 00:00:00 EDT 2016,Y,0,0,0
30477,10.625357,0.0,0.0,0.0,24,24.0000,0,0.0,0.0,0.0,0.0,321,-1,0.0,321.0,28,28.4297,0,0.0,0.0,0.0,0.0,4.0,2.4,Front-Wheel Drive,8,SIDI,6,2650,0,Premium,Premium Gasoline,6,-1,35,34.6595,0,0.0,0.0,0.0,0.0,0,0,37756,0,14,Acura,TLX FWD,N,False,0,93,0,0.0,0.0,0.0,0.0,Automatic (AM-S8),31.9972,0.0,50.4460,0.0,Compact Cars,2017,-1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HNX,NaN,0.0,NaN,Wed Jul 20 00:00:00 EDT 2016,Wed Apr 05 00:00:00 EDT 2017,N,0,0,0
30478,11.900400,0.0,0.0,0.0,21,21.0000,0,0.0,0.0,0.0,0.0,358,-1,0.0,358.0,25,25.0000,0,0.0,0.0,0.0,0.0,6.0,3.5,Front-Wheel Drive,9,SIDI,5,3000,0,Premium,Premium Gasoline,5,-1,34,34.1415,0,0.0,0.0,0.0,0.0,0,0

In [319]:
np.min([1.0, np.nan, 5.0])

nan

# Save to file

In [320]:
outdir = "./"

In [321]:
merged_all_the_data_yeah.to_csv(os.path.join(outdir, "2022_iihs_car_rankings_raw_DO_NOT_EDIT.csv"))

In [322]:
merged_all_the_data_yeah.to_excel(os.path.join(outdir, "2022_iihs_car_rankings.xlsx"), sheet_name="iihs car ranking")